In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os

class ImageProcessor:
    def __init__(self, image_path: str):
        self.image_path = image_path
        self.original_image = self._load_image()
        self.processed_images = {}

    def _load_image(self):
        if not os.path.exists(self.image_path):
            raise FileNotFoundError(f"Image not found at: {self.image_path}")
        
        img = cv2.imread(self.image_path)
        if img is None:
            raise ValueError(f"Could not load image from {self.image_path}. Check file format/corruption.")
        return img

    def save_image(self, img_data, output_filename="output.jpg"):
        cv2.imwrite(output_filename, img_data)
        print(f"Image saved as {output_filename}")

    def add_processed_image(self, name: str, img_data):
        self.processed_images[name] = img_data

    def resize_image(self, width: int, height: int):
        resized_img = cv2.resize(self.original_image, (width, height))
        self.add_processed_image("Resized", resized_img)
        return resized_img

    def crop_image(self, y_start: int, y_end: int, x_start: int, x_end: int):
        cropped_img = self.original_image[y_start:y_end, x_start:x_end]
        self.add_processed_image("Cropped", cropped_img)
        return cropped_img

    def convert_to_grayscale(self):
        gray_img = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2GRAY)
        self.add_processed_image("Grayscale", gray_img)
        return gray_img

    def convert_to_hsv(self):
        hsv_img = cv2.cvtColor(self.original_image, cv2.COLOR_BGR2HSV)
        self.add_processed_image("HSV", hsv_img)
        return hsv_img

    def apply_threshold(self, img_gray, threshold_val=120, max_val=255, type=cv2.THRESH_BINARY):
        _, thresh_img = cv2.threshold(img_gray, threshold_val, max_val, type)
        self.add_processed_image("Thresholded", thresh_img)
        return thresh_img

    def detect_edges(self, img_gray, low_thresh=100, high_thresh=200):
        edges_img = cv2.Canny(img_gray, low_thresh, high_thresh)
        self.add_processed_image("Edges", edges_img)
        return edges_img

    def find_and_draw_contours(self, img_thresh):
        contours, _ = cv2.findContours(img_thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        contour_img = self.original_image.copy()
        cv2.drawContours(contour_img, contours, -1, (0, 255, 0), 2)
        self.add_processed_image("Contours", contour_img)
        return contour_img

    def draw_shapes_and_text(self):
        draw_img = self.original_image.copy()
        cv2.line(draw_img, (0, 0), (200, 200), (255, 0, 0), 3)
        cv2.rectangle(draw_img, (50, 50), (150, 150), (0, 255, 0), 2)
        cv2.circle(draw_img, (100, 100), 40, (0, 0, 255), -1)
        cv2.putText(draw_img, "OpenCV", (20, self.original_image.shape[0] - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        self.add_processed_image("Drawing", draw_img)
        return draw_img

    def detect_faces(self, img_gray):
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
        faces = face_cascade.detectMultiScale(img_gray, 1.3, 5)
        face_img = self.original_image.copy()
        for (x, y, w, h) in faces:
            cv2.rectangle(face_img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        self.add_processed_image("Face Detection", face_img)
        return face_img

    def display_images_grid(self, max_cols=3, figsize=(15, 10)):
        num_images = len(self.processed_images)
        if num_images == 0:
            print("No images to display.")
            return

        rows = int(np.ceil(num_images / max_cols))
        fig, axes = plt.subplots(rows, max_cols, figsize=figsize)
        axes = axes.flatten() if rows * max_cols > 1 else [axes]

        for i, (title, img_data) in enumerate(self.processed_images.items()):
            ax = axes[i]
            if len(img_data.shape) == 2:
                ax.imshow(img_data, cmap='gray')
            else:
                ax.imshow(cv2.cvtColor(img_data, cv2.COLOR_BGR2RGB))
            ax.set_title(title)
            ax.axis("off")
        
        for j in range(i + 1, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    image_file = "gym_photo.jpg"

    if not os.path.exists(image_file):
        print(f"Creating a dummy image '{image_file}' for demonstration as it was not found.")
        dummy_img = np.zeros((300, 400, 3), dtype=np.uint8)
        cv2.putText(dummy_img, "Dummy Image", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.imwrite(image_file, dummy_img)
    
    processor = ImageProcessor(image_file)
    
    processor.add_processed_image("Original Image", processor.original_image)

    processor.save_image(processor.original_image, "gym_output.jpg")

    resized_img = processor.resize_image(300, 200)
    cropped_img = processor.crop_image(50, 250, 50, 350)

    gray_img = processor.convert_to_grayscale()
    hsv_img = processor.convert_to_hsv()

    thresholded_img = processor.apply_threshold(gray_img)
    edges_img = processor.detect_edges(gray_img)
    
    contours_drawn = processor.find_and_draw_contours(thresholded_img)

    drawn_shapes = processor.draw_shapes_and_text()
    
    face_detected_img = processor.detect_faces(gray_img) 

    processor.display_images_grid(max_cols=3, figsize=(18, 12))
